<a href="https://colab.research.google.com/github/AlexanderVinarsky/Muography_Research/blob/main/Muography_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
import numpy as np
import pandas as pd
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
df = np.load('drive/MyDrive/Muons/responses_norm.npy')
df = pd.DataFrame(df[-2])
df.style.background_gradient(cmap='plasma', axis=None)